# **Step1: Install and import libraries**

The first step is to load the data and learn about the characteristics of data.
Then we have to do some pre-proceesing on the text data in order to make it ready to be given to a model to learn.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install datasets
!pip install sentence_transformers
!pip install faiss-cpu
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Load dataset

In [ ]:
import os
from datasets import Dataset
import nltk
import evaluate

import torch
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import itertools
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from datasets import load_dataset
from datasets import load_metric

from sentence_transformers import SentenceTransformer, util
import faiss
from transformers import T5ForConditionalGeneration, T5Tokenizer

nltk.download("punkt", quiet=True)
rouge = load_metric("rouge")
bleu = load_metric("bleu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<ipython-input-3-ff9d5309038b>:23: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


# **Step2: Data Loading**

In [ ]:
dataset = load_dataset("eli5_category")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

## 2-1 Prepare dataset for train

In [ ]:
# Define weighted sampling function
def weighted_sample(answers, scores, k=3):
    k = min(k, len(answers))

    if len(answers) == 0:
        return []

    total_score = sum(scores)
    probabilities = [score / total_score for score in scores]
    sampled_indices = np.random.choice(len(answers), size=k, replace=False, p=probabilities)
    return [answers[i] for i in sampled_indices]

In [ ]:
def organize_dataset(dataset):
  '''
    input: dataset must be given as dataset['train'] or dataset['test']
  '''
  # Here, we just create a dictionary using the following columns
  #1)questions
  #2)self_text
  #3)categories
  #4)answers
  #5)answers_scores
  # Since other columns are unique acroos the whole dataset. If you would like to see wheater they are unique or not, you can
  # use the following commands
  print('Unique items in selftext_urls is just only: <', np.unique(dataset['selftext_urls'])[0],'> So, this column has been removed')
  print('Unique items in title_urls is just only: <', np.unique(dataset['title_urls'])[0],'> So, this column has been removed')
  print('Unique items in subreddit is just only: <', np.unique(dataset['subreddit'])[0],'> So, this column has been removed')


  questions = dataset['title']
  questions_selftext = dataset['selftext']
  categories = dataset['category']

  answers = []
  answers_scores = []
  for item in dataset:
    answers.append(item['answers']['text'])
    answers_scores.append(item['answers']['score'])

  dataset_dict = {"questions":questions, "selftext": questions_selftext, "category":categories, "answers":answers, "score":answers_scores}
  print('*'*100)
  print('Now, we have a clean datset dictionary with ',dataset_dict.keys())
  print('*'*100)
  return dataset_dict

In [ ]:
train_dataset_dict = organize_dataset(dataset['train'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [ ]:
valid1_dataset_dict = organize_dataset(dataset['validation1'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [ ]:
valid2_dataset_dict = organize_dataset(dataset['validation2'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [ ]:
test_dataset_dict = organize_dataset(dataset['test'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [ ]:
# Prepare data
train_questions = train_dataset_dict['questions']
train_answers = train_dataset_dict['answers']
train_scores = train_dataset_dict['score']

valid_questions = valid1_dataset_dict['questions']
valid_answers = valid1_dataset_dict['answers']
valid_scores = valid1_dataset_dict['score']

all_questions = train_dataset_dict['questions'] + valid1_dataset_dict['questions'] + valid2_dataset_dict['questions']
all_answers = train_dataset_dict['answers'] + valid1_dataset_dict['answers'] + valid2_dataset_dict['answers']
all_scores = train_dataset_dict['score'] + valid1_dataset_dict['score'] + valid2_dataset_dict['score']

# **Step3: Retrieval-Augmented Generation (RAG)**


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=0.1">
    
</head>
<body>
    <img src= "https://drive.google.com/uc?export=view&id=1YlcmgN2RA0vPTrvRRGMo_E-I4dFxk2pr" width="100%">

</body>
</html>

Retrieval-Augmented Generation (RAG) is an advanced approach that combines retrieval mechanisms with generative models to produce more accurate and comprehensive responses. This method leverages a retrieval component to fetch relevant information from a large corpus and then uses a generative model to construct a coherent and informative answer.


> **Retrieval Component**:
>
> This part of the system retrieves relevant documents or passages from a large corpus based on the input query.
>
> It typically uses models like BM25, TF-IDF, or dense retrieval models such as DPR (Dense Passage Retrieval) to find the most relevant texts.


> **Generation Component**:
>
>Once relevant documents or passages are retrieved, a generative model like T5 or GPT-3 is used to synthesize the information into a coherent and comprehensive answer.
>
>The generative model takes the retrieved texts and the query as inputs to produce the final output.

In this notebook, we are going to find the embeddings of sentences not words, so we will use SBERT instead of BERT embedding model.

Later we use the faiss package for indexing the embeddings and questions and at the end, for searching for similar questions for retrieving the context for questions (either at train phase or the evaluation part)

In [ ]:
# Load models
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

def find_Q_embeddings(questions):
    # Create a progress bar with the total number of steps as 1
    progress_bar = tqdm(total=1, desc="Encoding Questions")

    # Embed your dataset questions
    question_embeddings = sbert_model.encode(questions, convert_to_tensor=True)

    # Update the progress bar as the entire dataset is processed
    progress_bar.update(1)
    progress_bar.close()

    return question_embeddings

def index_Q_embeddings(question_embeddings):
    # Index embeddings using FAISS
    index = faiss.IndexFlatL2(question_embeddings.shape[1])
    index.add(question_embeddings.cpu().numpy())
    return index

def save_Q_embeddings(question_embeddings, question_embeddings_file='/kaggle/working/question_embeddings.npy'):
    # Save the embeddings to a file
    embeddings_file = 'question_embeddings.npy'
    np.save(question_embeddings_file, question_embeddings.cpu().numpy())
    print("Saved Question Embeddings in file.")
    pass

def load_Q_embeddings(train_question_embeddings_file='/kaggle/working/question_embeddings.npy'):
    # Load the embeddings from the file
    train_questions_embeddings = torch.tensor(np.load(train_question_embeddings_file))
    print("Loaded Question Embeddings.")
    return train_questions_embeddings


def find_relevant_context_SBERT(question, question_embeddings, answers, scores, index, use='train'):
    '''
    this function returns top-2 similar answers for training phase
    - top-2 answers of similar questions
    - top-2 answers of current question
    '''
    new_question_embedding = sbert_model.encode(question, convert_to_tensor=True, show_progress_bar=False).unsqueeze(0)
    new_question_embedding = torch.nn.functional.normalize(new_question_embedding, p=2, dim=1)

    k = 5  # Retrieve a larger number of candidates
    D, I = index.search(new_question_embedding.cpu().numpy(), k)

    threshold = 0.5
    filtered_indices = []
    for i, distance in zip(I[0], D[0]):
        candidate_embedding = question_embeddings[i].cpu().numpy().reshape(1, -1)
        similarity = cosine_similarity(new_question_embedding.cpu().numpy(), candidate_embedding)[0][0]
        if similarity >= threshold:
            filtered_indices.append(i)

    if (len(filtered_indices) == 0):
        if(use == 'test'):
            return [""], [""]
        else:
            n_index = all_questions.index(question)
            top_ans = weighted_sample(answers[n_index], scores[n_index], k=3)
            return top_ans, [""]
    else:
        # Retrieve relevant contexts (questions and answers)
        relevant_contexts = [weighted_sample(answers[i], scores[i], k=3) for i in filtered_indices[1:]]
        if len(relevant_contexts) >= 1:  # when no relevant context is found
            relevant_contexts = relevant_contexts[0]
        top_2_ans = weighted_sample(answers[filtered_indices[0]], scores[filtered_indices[0]], k=3)
        return top_2_ans, relevant_contexts

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 3-1 Create SBERT embeddings for all questions

In [ ]:
# question_embeddings = find_Q_embeddings(all_questions)
# save_Q_embeddings(question_embeddings, question_embeddings_file='/kaggle/working/question_embeddings.npy')

## 3-2 Create faiss index on questions for question-question similarities

In [ ]:
# loaded_question_embeddings = load_Q_embeddings("/kaggle/working/question_embeddings.npy")
question_embeddings = load_Q_embeddings("/content/drive/MyDrive/Colab Notebooks/LFQA final files/question_embeddings.npy")
faiss_index = index_Q_embeddings(question_embeddings)

Loaded Question Embeddings.


In [ ]:
# Example usage
question = 'Why shouldnt we laugh at people?'
question = "how is the earth round?"
# question = train_dataset_dict['questions'][0]
# question = test_dataset_dict['questions'][1]
# answers = test_dataset_dict['answers'][1]
# question = train_dataset_dict['questions'][4]
# answers = train_dataset_dict['answers'][4]
question = valid_questions[0]
question = "What is a NIMBY?"

# context = retrieve_contexts(model_bert, tokenizer_bert, train_dataset_dict, question, train_question_embeddings, index=None, top_n=8)
print("\n question:")
print(question)
print("\n answers:")
# print(answers)

cntx = find_relevant_context_SBERT(question, question_embeddings, all_answers,
                                   all_scores, faiss_index)
print("\n context")
print(cntx)


 question:
What is a NIMBY?

 answers:

 context
(["Not in my Backyard. The name is pretty self-explanatory, but it's usually something that provides a net benefit to a given area but inconveniences one person or a small group of people. The first example that comes to mind would be something like a cell tower, undesirable business, or a noisy highway.", 'Not in my back yard. An acronym for things that are needed but not wanted close by. Prison, garbage dump, Nuclear Power plant etc.', 'Not in my Backyard. It\'s referring to people who want all sorts of public services, but at the same time oppose building it anywhere near their homes. For example, they want highways so they can get to work faster, but they don\'t want a highway anywhere near their home. They want great public transit, but they don\'t want a train station built down the block. They also don\'t want buses going down their street every day. They want an airport that is close to their home so it\'s easy to travel, but th

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # RougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds], references=[[label.split()] for label in decoded_labels])

    return {**rouge_result, **bleu_result}

# **Step4: Define T5 model for fine-tuning**




<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=0.1">
    
</head>
<body>
    <img src= "https://drive.google.com/uc?export=view&id=1hEN74geV8tNC6H_OJyoo2ZPRlGzpxTzV" width="100%">

</body>
</html>


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import random

# Load the T5 tokenizer and model
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.to(device)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Encoding Questions:   0%|          | 0/1 [06:17<?, ?it/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

## 4-1 Prepare data for training

Lets create a custome dataset to be used for creating the appropriate prompt for training the T5 model having the question, the retrieved context adn its top-score answer.

In [ ]:
# Custom Dataset Class
class DynamicQADataset(Dataset):
    def __init__(self, questions, answers, scores, question_embeddings, faiss_index,
                 tokenizer, epoch, max_length=1024):
        self.questions = questions
        self.answers = answers
        self.scores = scores
        self.tokenizer = tokenizer
        self.epoch = epoch
        self.max_length = max_length
        self.question_embeddings = question_embeddings
        self.faiss_index = faiss_index

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answers = self.answers[idx]
        scores = self.scores[idx]

        top_2_ans, relevant_contexts = find_relevant_context_SBERT(question,
                                                                         self.question_embeddings,
                                                                         self.answers,
                                                                         self.scores, self.faiss_index)

        best_answer = top_2_ans[min(self.epoch, len(top_2_ans)-1)]
        if (len(top_2_ans) > 1):
            del top_2_ans[self.epoch]
        cntx = relevant_contexts + top_2_ans

        input_text = f"question: {question} context: {' '.join(cntx)}"
        target_text = best_answer

        # Tokenize the inputs and targets
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs.input_ids.squeeze(0)
        attention_mask = inputs.attention_mask.squeeze(0)
        labels = targets.input_ids.squeeze(0)

        # Replace padding token id's of the labels by -100 so it's ignored by the loss
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Function to create a DataLoader for a specific epoch
def create_dataloader(questions, answers, scores, question_embeddings, faiss_index, epoch, batch_size=2):
    dataset = DynamicQADataset(questions, answers, scores, question_embeddings, faiss_index, tokenizer, epoch)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
def decode_predictions(preds, labels, tokenizer):
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return decoded_preds, decoded_labels


## 4-2 Training

In [ ]:
# Training parameters
epochs = 2
batch_size = 1
learning_rate = 5e-5
adam_epsilon = 1e-8
warmup_steps = 0
checkpoint_interval = 20000  # Save a checkpoint every 10000 batches

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
total_steps = len(train_questions) // batch_size * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

checkpoint_dir = "/kaggle/working/"

## 4-3 Load model from last saved checkpoint

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import random

# Load the T5 tokenizer and model
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
total_steps = len(train_questions) // batch_size * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    checkpoint = torch.load(checkpoint_path,  map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # start from the next epoch
    loss = checkpoint['loss']
    return model, optimizer, scheduler, start_epoch, loss

# Initialize variables
start_epoch = 0  # default start epoch
checkpoint_path = os.path('/content/drive/MyDrive/Colab Notebooks/LFQA final files/checkpoint_epoch_1_batch_40000.pt')
if os.path.exists(checkpoint_path):
    model, optimizer, scheduler, start_epoch, loss = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
    print(f"Loaded checkpoint from epoch {start_epoch}, loss: {loss}")


Loaded checkpoint from epoch 2, loss: 75456.31543746311


## 4-4 Training Loop

In [ ]:

# Training loop
model.train()
for epoch in range(epochs-1):
    val_loader = create_dataloader(valid_questions, all_answers, all_scores, question_embeddings, faiss_index, epoch, batch_size)
    train_loader = create_dataloader(train_questions, all_answers, all_scores, question_embeddings, faiss_index, epoch, batch_size)
    total_loss = 0

    for batch_idx, batch in enumerate(tqdm(train_loader, desc="Batch progress")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Checkpointing after every checkpoint_interval batches
        if (batch_idx + 1) % checkpoint_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}_batch_{batch_idx+1}.pt')
            torch.save({
                'epoch': epoch + 1,
                'batch_idx': batch_idx + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': total_loss
            }, checkpoint_path)
            print(f'Saved checkpoint_epoch_{epoch+1}_batch_{batch_idx+1}')

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

    # Checkpointing after each epoch
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': total_loss
    }, checkpoint_path)

    # Validation
    model.eval()
    val_loss = 0
    eval_preds = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation progress"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Collect predictions and labels for metric computation
            preds = outputs.logits.argmax(dim=-1)
            eval_preds.extend(zip(preds.cpu().numpy(), labels.cpu().numpy()))

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss}")

    model.train()

## 4-5 Saving Model

In [ ]:
# save model and tokenizer
model.save_model("/kaggle/working/RAG_SBERT_faiss_T5")
tokenizer.save_pretrained("/kaggle/working/RAG_SBERT_faiss_T5")

In [ ]:
# !cp ./kaggle/ /content/drive/MyDrive/LFQA\ final\ files/RAG_SBERT_faiss_T5

## 4-6 Load Model

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import random

model_directory = '/content/drive/MyDrive/LFQA final files/checkpoint_epoch_1.pt' # Set this to your directory

tokenizer = T5Tokenizer.from_pretrained(model_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory)

model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2004: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


# **Step5: Generating answer**




Now lets try a new question. what we do is that try to find a context based on similar questions in our dataset and then generate an asnwer with that context using the trained model.

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=0.1">
    
</head>
<body>
    <img src= "https://drive.google.com/uc?export=view&id=1tCr-prcSjbnyP9RHgHSU0UT5kpoZXv6M" width="100%">

</body>
</html>

In [ ]:
def generate_answer(question, context):
    input_text = f"question: {question} context: {' '.join(context[0] + context[1])}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    outputs = model.generate(inputs, min_length=100, max_length=1024, num_beams=5, early_stopping=True).to(device)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Example usage
# question = 'how to talk Korean?'
# question = train_questions[150]
# question = valid_questions[20]
# question = "What is NIMBY?"
# question = "What is pedophilia?"
question = test_dataset_dict['questions'][8]
print(question)

How do DSLR cameras work?


In [ ]:
cntx = find_relevant_context_SBERT(question, question_embeddings, all_answers, all_scores, faiss_index, use='test')
answer = generate_answer(question, cntx)
print(answer)

Your camera is just a sensor that takes in light. The storage system on your camera is a small computer. When you take a single photo, it takes in all that light for that moment and stores it as a photo (technically just as a string of colors, not even as a photo tbh)... Your viewer app looks at the string and forms it into the photo. For video, it just assumes you didn't want to stop. It keeps sucking in light and storing it until you say stop or you run out of space. The app reads the string of colors and decides that based on the size you wanted (HD or 4K or something), that one color is the end of a picture and the next is the next.
